In [26]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset


dataset = load_dataset('phihung/titanic')
train_data = pd.DataFrame(dataset['train'])

In [27]:
import numpy as np

bins = [-np.inf, 6, 16, 36, 55, np.inf]
labels = ['Baby', 'Child', 'Youth', 'Adult', 'Senior']

train_data['AgeGroup'] = pd.cut(train_data['Age'], bins = bins, labels = labels)

In [28]:
pclass_mean1 = train_data['Fare'][train_data['Pclass'] == 1].mean()
pclass_mean2 = round(train_data['Fare'][train_data['Pclass'] == 2].mean(), 4)
pclass_mean3 = round(train_data['Fare'][train_data['Pclass'] == 3].mean(), 4)

train_data.loc[(train_data['Pclass'] == 1) & (train_data['Fare'] == 0), 'Fare'] = pclass_mean1
train_data.loc[(train_data['Pclass'] == 2) & (train_data['Fare'] == 0), 'Fare'] = pclass_mean2
train_data.loc[(train_data['Pclass'] == 3) & (train_data['Fare'] == 0), 'Fare'] = pclass_mean3

In [29]:
train_data['FareGroup'] = pd.cut(train_data['Fare'], bins = [0, 41, 101, 201, np.inf], labels=[1, 2, 3, 4]).astype(int)

In [30]:
def titling(X, attrib):
    titles = []
    for name in X[attrib]:
        if type(name) == str:
          # Split the name by commas, and extract the title (which is after the comma and before the period)
          # Allen, Mr. William Henry
          title = name.split(',')[1].split('.')[0].strip()
          titles.append(title)
        else:
          titles.append('')

    return titles

def tilgrouping(X, attrib):
    royal = ['Mme', 'Lady', 'Mlle', 'Col']
    common = ['Mr', 'Mrs', 'Miss', 'Dr', 'Ms', 'Master','Ms']
    low = ['Don', 'Rev', 'Capt', 'Jonkheer', '']

    titleg = []

    for i in range(len(X[attrib])):
        if (X[attrib][i]) in royal:
            titleg.append(1)
        elif (X[attrib][i]) in common:
            titleg.append(2)
        else:
            titleg.append(3)

    return titleg

In [31]:
train_data['Title'] = titling(train_data, 'Name')
train_data['TitleGroup'] = tilgrouping(train_data, 'Title')

In [32]:
train_data['FamilySize'] = train_data['SibSp'] + train_data['Parch']
train_data['FamilyType'] = pd.cut(train_data['FamilySize'], bins=[-1, 0.5, 4, np.inf], labels=['Alone', 'Small', 'Big'])

In [33]:
dummies = pd.get_dummies(train_data[['Sex', 'Embarked', 'AgeGroup']], drop_first=True)
train_data = pd.concat([train_data.drop(['Sex', 'Embarked', 'AgeGroup'], axis=1), dummies], axis=1)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train_data['FamilyType'])
train_data['FamilyType'] = le.transform(train_data['FamilyType'])

In [37]:
train_data.head()


,PassengerId,Survived,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,...,TitleGroup,FamilySize,FamilyType,Sex_male,Embarked_Q,Embarked_S,AgeGroup_Child,AgeGroup_Youth,AgeGroup_Adult,AgeGroup_Senior
0,1,0,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,None,...,2,1,2,True,False,True,False,True,False,False
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,...,2,1,2,False,False,False,False,False,True,False
2,3,1,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,None,...,2,0,0,False,False,True,False,True,False,False
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,...,2,1,2,False,False,True,False,True,False,False
4,5,0,3,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,None,...,2,0,0,True,False,True,False,True,False,False


In [39]:
train_data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'FareGroup', 'Title', 'TitleGroup',
       'FamilySize', 'FamilyType', 'Sex_male', 'Embarked_Q', 'Embarked_S',
       'AgeGroup_Child', 'AgeGroup_Youth', 'AgeGroup_Adult',
       'AgeGroup_Senior'],
      dtype='object')

In [ ]:
train_data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'Title'], axis=1, inplace=True)

In [36]:
from sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler()
datatrmd = scaler.fit_transform(train_data)
train_data = pd.DataFrame(data=datatrmd, columns=train_data.columns)
train_data.head()

ValueError: could not convert string to float: 'Braund, Mr. Owen Harris'

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(train_data[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch',
                                                                 'Fare', 'Embarked', 'AgeGroup', 'FareGroup', 'TitleGroup', 'FamilySize', 'FamilyType']], train_data[['Survived']], test_size=0.3, random_state=42)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Create an instance of the DecisionTreeClassifier
tree = DecisionTreeClassifier()

# Fit the decision tree model to the training data
tree.fit(X_train, y_train)

# Predict the target variable for the validation data
y_pred = tree.predict(X_valid)


ValueError: could not convert string to float: 'male'